In [1]:
import pandas as pd
import numpy as np
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from string import punctuation
import pymorphy2
import math

In [2]:
stop = stopwords.words('russian') 
pymorph = pymorphy2.MorphAnalyzer()
def q_preprocess(query):
    query = query.replace(r"[^\w]", " ")
    query = query.replace(r"[\d]", " ")
    query = query.lower()
    filtered_sentence = ''
    query = nltk.word_tokenize(query)
    for w in query:
        if w not in stop:
            filtered_sentence = filtered_sentence + ' ' + pymorph.parse(w)[0].normal_form
    return filtered_sentence

In [3]:
df = pd.read_csv('dataset.csv', encoding = 'utf8')
df.describe()

,tokens,name,text,author,cat,url
count,25026,25026,25026,25026,25026,25026
unique,25026,24391,25026,9045,20403,25026
top,этот книга десятилетие определять судьба чело...,Возвращение,"Эта книга о десятилетии, определяющем судьбу ч...",Неустановленный автор,Современные детективы;,https://mybook.ru/author/meg-dzhej/vazhnye-god...
freq,1,13,1,471,130,1


In [4]:
documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(df['tokens'])]

In [5]:
model = Doc2Vec(documents, vector_size=50, min_count=2, epochs = 80)

In [205]:
model.save("doc2vec.model")

In [274]:
vector = model.infer_vector(q_preprocess('герой спасает принцессу из замка').split())

In [253]:
sims = model.dv.most_similar(vector, topn=5)
print(sims)

[(1095, 0.6513590812683105), (24056, 0.6379069685935974), (504, 0.6322513818740845), (10208, 0.6310149431228638), (12696, 0.6281949281692505)]


In [6]:
df['text'][1095]

'Не каждая Золушка мечтает стать принцессой. Иногда бывает и по-другому. Ты – хозяйка трактира, рядом друзья, и жизнь идет своим чередом… И ты не ведаешь и не гадаешь, что сказка уже совсем рядом. Когда жизнь наполнена любимым делом, а сердце – верой в добро и заботой о людях, то счастье обязательно постучится в двери, даже если его не ждешь. В любом мире и времени найдется своя Золушка, ведь больше всего люди любят… любовь!'

### Тестирование

In [ ]:
def otiai_similarity(list1, list2):
    intersection = len(list(set(list1).intersection(list2)))
    sqrt = math.sqrt((len(list1) * len(list2)))
    return float(intersection) / sqrt

In [307]:
%%time
vector = model.infer_vector(df['tokens'][857].split())
sims = model.dv.most_similar(vector, topn=2)
sims

Wall time: 4.53 ms


[(857, 0.9792442321777344), (1068, 0.9406984448432922)]

In [306]:
jcds = np.array([])
for i in range(1000):
    inferred_vector = model.infer_vector(documents[i].words)
    sims = model.dv.most_similar([inferred_vector], topn=2)
    if sims[1][1] > 0.8:
        jd = otiai_similarity(df['cat'][i].split(';'), df['cat'][sims[1][0]].split(';'))
        jcds = np.append(jcds, jd)
print(sum(jcds)/len(jcds))
print(len(jcds))

0.737114896342883
15
